# <span style="color:cornflowerblue"> Gerald Jones, Kimon Swanson, Alex Hines</span>
# <span style="color:cornflowerblue"> Home Work 5: Farmers Problem</span>
# <span style="color:cornflowerblue"> ISE522 Spg 22</span>

## Notebook Links:
1. [Data Display section](#Data-Display)
2. [Method Definitions](#Method-Definitions)
3. [Model Formulation Problem 1](#Model-Formulation-1)
4. [Gurobi Implementation Problem 1](#prob1)
5. [Solution Discussion Problem 1](#solution1)
6. [Model Formulation Problem 2](#Model-Formulation-2)
7. [Gurobi Implementation Problem 2](#prob2)
8. [Solution Discussion Problem 2](#solution2)
9. [Model Formulation Problem 3](#Model-Formulation-3)
10. [Gurobi Implementation Problem 3](#prob3)
11. [Solution Discussion Problem 3](#solution3)

## Problem Description:
> The farmer's problem Consider and farmer in east Tennessee who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop. The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler. Any production in excess of the feeding requirement would be sold. Over the last decade, mean selling prices have been $\text{\$170}$ and $\text{\$150}$ per ton of wheat and corn, respectively. The purchase prices are 40% more than this due to the wholesaler's margin and transportation costs.
<br><p>
> Another profitable crop is **sugar beet**, which he expects to sell at $\text{\$36}$/T; however, the EU imposes a quota on sugar beet production. Any amount in excess of the quota can be sold onlv at $\text{\$10}$/T. The farmer'squota for next year is 6,000 T. Based on past experience, the farmer knows that he mean yield on his land is roughly 2.5 T,3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. The table on the next slide summarizes these data and provides the planting costs for these crops. Formulate a linear program to maximize total profit (planting cost —revenue from sales).</p>

# Extentions To Accomplish:
1. [Suppose the farmer allocates 120 acres to wheat, 80 to corn, and 300 for sugar beets. But also suppose that the 
   yields are not certain but are subject to the same 3 scenarios we discussed in class (above average, average, below 
   average, each with equation probability. What is the expected annual profit?](#prob1)
    - [Problem 1 Method Defintion and Execution Cell](#prob1-meth)
2. [Consider an extension of the problem in which prices also are uncertain. Specifically, when yields are above 
   average prices for corn and wheat go down by 10% and when yields are below average, they go up by 10%.
   Assume sugar beet prices are not affected by yields. Formulate and solve a stochastic linear program and solve 
   it with a solver.](#prob2)
3. [Consider a different version of the problem in which the farmer possesses four fields of sizes 185, 145, 105, and 
   65, respectively. The total of 500 acres is unchanged. However, the farmer wishes to only plant one type of crop 
   on each field. Formulate and solve a stochastic program and solve it with a solver](#prob3)

## Notes and Observations
* Total of 500 acres

* 3 possible crops:
    - wheat 
        + (>= 200 T through crops or purchase)
        + $\text{\$170}$/T selling price
        + $\text{\$170}$*40  + $\text{\$170}$ purchase price = $\text{\$238}$/T
        + AVG yield: 2.5 T
    - corn  
        + (>= 240 T through crops or purchase)
        + $\text{\$150}$/T selling price
        + $\text{\$150}$*40 + $\text{\$150}$ purchase price = $\text{\$210}$/T
        + AVG yield: 3 T
    - sugar beats
        + selling price is tiered
            - below quota:= $\text{\$36}$/T
            - above quota:= $\text{\$10}$/T
        + quota:= 6000 T
        + AVG yield: 20 T

* any purchase of wheat or corn over the required amount is sold for profit at indicated selling price

## Assumptions:


# <span style="color:orange"><center><b>Module imports and data loading</b></center></span>

In [1]:
from _GUROBI_TOOLS_.GUROBI_MODEL_BUILDING_TOOLS import *
from _NOTE_BOOK_UTILS import *
import numpy as np

# name for the conversion to pdf method from _NOTE_BOOK_UTILS
notebook_title = "_HW5_Problem3.ipynb"

<IPython.core.display.Javascript object>

# <a id=Data-Display><span style="color:Green"><center> Data Display & Python Variable Assignments</center></span></a>

## Below cell defines several variables representing the data and constraints 

In [2]:
# display data for problem
# data is represented by the instantiated dataframe show here and displayed in the output below

# data for the problem
data_df = pd.DataFrame(
{
    "Wheat":[2.5, 150, 170, 170, 238, 200], 
    "Corn": [3,   230, 150, 150, 210, 240],
    "Sugar Beets":[20, 260, 36, 10, 0, 0],
},
    index=["Yield", "Planting Cost", "Selling Cost1", "Selling Cost2", "Price", "min"]
)

display(data_df)

# used to calculate the above/below average yields
average_rate = .2 # (percentage above/below the respective high and low average scenerios)

# price percentage difference in the low and high average yield scenerios 
price_rate = .1



#######################################################################################
# Initial amount of acreage devoted to each crop type
crop_acreage = [
    120, # acres of wheat to plant
    80,  # acres of corn to plant
    300  # acres of sugar beets to plant
]

print("Initial Allotted Acreage:")
print("Wheat: {}".format(crop_acreage[0]))
print("Corn: {}".format(crop_acreage[1]))
print("Sugar Beets: {}".format(crop_acreage[2]))

# adjustment to yield rate based on scenerio
# rows-> crops, columns->scenerio
# In this problem the scenerios are expected to see a 20% decrease, or increase 
# for the below and above average cases respectively
crop_yieldRate_scenerio = [-.2, 0, .2]  # below average, average, above average-->wheat
               

crop_price_scenerio =[-.1, 0, .1]  # below average, average, above average-->wheat


feedCrops = ["Wheat","Corn"]
# Remove this it is not needed the information is in the DF
# Minimum amount of wheat/Corn that must be yielded for feed crop purposes
# minT_dict = {
#     "Wheat":200,        # minimum tons of wheat needed for feed
#     "Corn":240,         # minimum tons of corn needed for feed
# }

# 
print("\t\t\t------------------------- Minimum crop resources for feed crops")
for c in feedCrops:
    print("Require a minimum of {} tons of {}".format(data_df.loc["min",c], c))
print("-------------------------------------------------")
# print(ysc)
# print(len(ysc))
quota = 6000               # yield level that when exceeded causes price of sugar beets-->reduced/penalized

# Probabilities for he below, average, and above average cases
scenerio_probs = [
    1/3,  # below average yield probability-->
    1/3,  # average yield probability
    1/3,  # above average yield probability
]

# field sizes:  0    1    2    3
field_sizes = [185, 145, 105, 65]


,Wheat,Corn,Sugar Beets
Yield,2.5,3,20
Planting Cost,150.0,230,260
Selling Cost1,170.0,150,36
Selling Cost2,170.0,150,10
Price,238.0,210,0
min,200.0,240,0


Initial Allotted Acreage:
Wheat: 120
Corn: 80
Sugar Beets: 300
			------------------------- Minimum crop resources for feed crops
Require a minimum of 200.0 tons of Wheat
Require a minimum of 240 tons of Corn
-------------------------------------------------


# <a id=prob3>Problem 3</a>:

[]() | []()
> 3. Consider a different version of the problem in which the farmer possesses four fields of sizes 185, 145, 105, and 
   65, respectively. The total of 500 acres is unchanged. However, the farmer wishes to only plant one type of crop 
   on each field. Formulate and solve a stochastic program and solve it with a solver
## Notes:
> This is a selection problem. We need to decide which crop to grow where so that we maximize profits and minimize purchasing costs as well as trying to keep the minimum feed crop and sugar beet constraints intact.: 
* Need Binary selector variables
    + need 3 for each of the 4 fields
        - need 

# <a id=Model-Formulation-3><center> <span style="color:blue"> Model Formulation Problem 3</span> </center></a>
* [Paremeters and Sets](#Parameters-and-Sets-3)
* [Variables](#Variables-3)
* [Equations and Constraints](#Equations-and-Constraints-3)
* [Objective](#Objective-3)

## <a id=Parameters-and-Sets-3><span style="color:DarkBlue">Indices, Parameters, and Sets 3:</span></a>

### $\textbf{C}  \quad \quad \quad \text{set of possible crops {wheat(1), corn(2), sugar-beets $\leq$ 6KT(3), sugar-beets > 6KT (4)} } c \in \textbf{C}$ 
### $F \quad \quad \quad \text{set of possible planting fields,  \{1, 2, 3, 4\}} f\in F$
### $D_{f,c} \quad \quad \text{binary variable, 1 if field f will have crop c planted, 0 otherwise}$
### $X_{f,c} \quad \quad \text{acres allocated to crop c, in field f, } c \in\ \{wheat(1), corn(2), sugarbeets(3)\}$
### $Y_{c } \quad \quad \text{ tons purchased for crop c $\in$ \{1, 2\}}$
### $E_{c} \quad \quad \text{ Expected yield (tons) for crop c in tons}$
### $W_{c} \quad \quad \text{ tons of crop c sold}$
### $C_{c} \quad \quad \text{ purchase cost for crop } c \in \{wheat, corn\}$
### $L_{c} \quad \quad \text{ planting cost ($\$$/acre) for crop } c \in C$
### $P_{c} \quad \quad \text{ sale price for crop c}$
### $\mu_{c} \quad \quad \text{ minimum amount of feed crop c} \in \{1, 2\}$
### $A_{f} \quad \quad \text{ maximum amount of acreage available for field f}$
### $\rho_{c} \quad \quad\text{ average expected ton/acre yield rate for crop c}$
### $\mathbb{G} \quad \quad \text{Total Planting costs}$
### $\mathbb{R} \quad \quad \text{Total Purchasing costs for feed crops}$
### $\mathbb{H} \quad \quad  \text{Total sales profits from all excess crops}$
### $M \quad \quad \text{large number}$

## <a id=Equations-and-Constraints-2><span style="color:DarkBlue">Equations and Constraints:</span></a>

>### <center><span style="font-size:30px;color:red"><b>Equation </b></span></center>

# First Stage: How much to plant
> ### Nonzero acreage constraint: $X_{f,c} \geq 0, \text{ }\forall f,c$
> ### Nonzero acreage constraint: $X_{f,c} \leq M \cdot D_{f,c}, \text{ }\forall f,c$
> ### Binary Planting Decision variables: $\sum_{c=1}^{3}(D_{f,c}) \leq 1, \text{ }\forall f$
> ### Maximum Acreage Available: $\sum_{f=1}^{4}\sum_{c}^{|C|}(X_{f,c}) \leq 500$
> ### Total Planting Costs Expression: $\mathbb{G} = \sum_{f=1}^{4}\sum_{c=1}^{3}(X_{f,c} \cdot L_c)$
> ### Expected Yield for a given crop: $E_c = \sum_{f=1}^{4}(\rho_c \cdot X_{f,c}), c \in \{1,2,3\}$
> ### Purchasing Costs for crop c: $C_c = E_c \cdot P_c$
> ### Total purchasing costs: $\mathbb{R} = \sum_{c=1}^{2}(C_c), c \in \{1,2\}$
> ### Total Sales Profits costs: $\mathbb{H} = \sum_{c=1}^{4}W_c \cdot P_c$

# General Constraints

> ### Minimum feed crop constraints: $Y_{c} + E_{c} - W_{c} \geq \mu_c, \forall c \text{ where }c \in \text{ \{wheat(1), corn(2)\}}$

> ### Sugar Beets Sales constraint: $E_{3} - \sum_{c=3}^{4}W_{c} \geq 0, \forall s$
> ### Sugar Beet below quota value constraint: $ W_{3} \leq 6000, \forall s$



## <a id=objective-2><span style="color:green">Obective:</span></a>
> ## Maximize: $\mathbb{H}- \mathbb{R} - \mathbb{G}$

# <a id=prob2-gurobi><center>Gurobi Implementation and Solution Problem 2</center></a>

In [3]:
try:
    # instantiate model object 
    m = gp.Model("Farmer_problem3")
 
    
    
    #########################################################################################
    ################################## Parameters set up ####################################
    #########################################################################################
    C = 3       # number of crop lots 
    S = 3       # number of possible scenerios
    L = 4       # number of different lots for planting
    F = len(feedCrops)        # amount of feed crops
    A = 500
    display(data_df)
    crop_yieldRate_scenerio = [.8, 1, 1.2]  # below average, average, above average
    crop_price_scenerio =[1.1, 1, .9]  # below average, average, above average
    #########################################################################################
    ################################## Variables set up #####################################
    #########################################################################################
    Xfc = m.addVars(L, C, vtype=GRB.CONTINUOUS, name="X", lb=0)  # acreage per crop for planting
    Dfc = m.addVars(L, C, vtype=GRB.BINARY, name="D", lb=0)      # Decision Variable for which plot to plant what crop
    Ec = m.addVars(C, vtype=GRB.CONTINUOUS, name="E", lb=0,)  # expected yield for each crop & scenerio
    Yc = m.addVars(F, vtype=GRB.CONTINUOUS, name="Y", lb=0)   # amount to purchase for feed crops
#     Cc = m.addVars(F, vtype=GRB.CONTINUOUS, name="C", lb=0)   # purchase costs for feed crops & scenerio
    Wc = m.addVars(C+1, vtype=GRB.CONTINUOUS, name="W", lb=0) # amount to sell per crop
    
    G =   m.addVars(1, vtype=GRB.CONTINUOUS, name="G", lb=0,)     # planting costs
    H = m.addVars(1, vtype=GRB.CONTINUOUS, name="H", lb=0) # profits for sales in each scenerio for each crop
    R = m.addVars(1, vtype=GRB.CONTINUOUS, name="R", lb=0) # purchase costs for each scenerio for feed crops
    M = A*1.5                            # large number is just the total lot space times a half

    # get list of the crop names
    crops = data_df.columns.to_list()

    #########################################################################################
    ################################## Objective set up #####################################
    #########################################################################################    
    # set up expected yields for each crop over all fields
    for c in range(C):
        tot_crop_c = 0
        for l in range(L):
            tot_crop_c += Xfc[l, c]
        # set this crops total Estimated yield to be the total acreage it was given time
        # the expected yield rate
        m.addConstr(Ec[c] == data_df.loc["Yield", crops[c]]*tot_crop_c)

    # set up planting costs (G)
    exprs = 0
    for l in range(L):
        for c in range(C):
            print(data_df.loc["Planting Cost", crops[c]])
            exprs += Xfc[l, c]*data_df.loc["Planting Cost", crops[c]]
    m.addConstr(G[0]==exprs)
    
    # add big M constraint for decision variable usage
    # for each of the possible crop selections
    for l in range(L):
        for c in range(C):
            m.addConstr(Xfc[l, c] <= M*Dfc[l,c])
    
    # add singular lot to crop assignment constraint
    for l in range(L):
        lot_binary_total = 0
        # for each lot sum all decision variables and make sure they do not go above 0
        for c in range(C):
            lot_binary_total += Dfc[l, c]
        m.addConstr(lot_binary_total <= 1)
    
    # maximum acreage constraint
    tot_acres = 0
    for l in range(L):
        for c in range(C):
            tot_acres += Xfc[l,c]
    m.addConstr(tot_acres <= A)
    
    # lot size constraints for each lot
    for l in range(L):
        lot_size = 0      # initial total lot size to zero
        # for each possible crop sum up the amount for this lot
        for c in range(C):
            lot_size += Xfc[l, c]
        # add the constraint that the total acres alloted 
        # can not exceed what is available
        m.addConstr(lot_size <= field_sizes[l])
        
        
    
    # ######################                           Feed Crop Constraints
    # set up purchasing cost for different scenerios

    
    # set up minimum feed crop constraints
    m.addConstrs(Yc[c] - Wc[c] + Ec[c] >= data_df.loc["min", crops[c]] for c in range(F))
   
    # set up purchasing costs expressions for each scenerio
    exp = 0
    for c in range(F):
        exp += Yc[c]*data_df.loc["Price", crops[c]]
    
    m.addConstr(R[0]==exp)
   
    exp = 0
    # set up the total profits expression for the feed crops
    for c in range(2):
        exp += Wc[c]*data_df.loc["Selling Cost1", crops[c]]
    
    # add expressions for the sugar beets at the different cost levels
    exp += Wc[2]*data_df.loc["Selling Cost1", crops[2]] + Wc[3]*data_df.loc["Selling Cost2", crops[2]]
    m.addConstr(H[0]==exp)

       
        
    # ######################                             Sugar Beet Constraints
    m.addConstr(Ec[2] - Wc[2] - Wc[3] >= 0)   # set up sugar beet amounts constraint
    m.addConstr(Ec[2] <= quota)                # set up lower level of quota sales constraint
    
    # get each scenerios purchasing costs and sales profits
    
    
    #########################################################################################
    ################################## Constraint set up ####################################
    #########################################################################################
    m.setObjective(H[0] - R[0] - G[0], GRB.MAXIMIZE)
    
    #########################################################################################
    ################################## SOLVE:OPTIMIZE #######################################
    #########################################################################################    
    ## Objective should be for(H(s) - R(s))
    
    m.optimize()
    
    #########################################################################################
    ################################## Display Results ######################################
    #########################################################################################    
    displayDecisionVars(m, end_sentinel="10")
    
    print("\n-------------Does it make sense?----------------------")  
    print('Obj: {:.2f}'.format(m.ObjVal))
    
    
# catch some math errors
except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Restricted license - for non-production use only - expires 2023-10-25


,Wheat,Corn,Sugar Beets
Yield,2.5,3,20
Planting Cost,150.0,230,260
Selling Cost1,170.0,150,36
Selling Cost2,170.0,150,10
Price,238.0,210,0
min,200.0,240,0


150.0
230
260
150.0
230
260
150.0
230
260
150.0
230
260
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 31 rows, 36 columns and 106 nonzeros
Model fingerprint: 0x922aa527
Variable types: 24 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+03]
Found heuristic solution: objective -30400.00000
Presolve removed 12 rows and 14 columns
Presolve time: 0.00s
Presolved: 19 rows, 22 columns, 54 nonzeros
Variable types: 14 continuous, 8 integer (8 binary)

Root relaxation: objective 1.186000e+05, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 118600.000    0    2 -30400.000 118600.000   490% 

# <a id=solution3><span style="color:crimson"><center>Solution 3 Exploration & Discussion</center></a>

# Solution Exploration and Analysis

# acreage Decisions:
* Lot 1:
    - X[0,2] 185.00000  --> Sugar Beets
* Lot 2: 
    - X[1,0] 145.00000   --> Wheat 
* Lot 3: 
    - X[2,2] 105.00000  --> Sugar beets
* Lot 4: 
    - X[3,1] 65.00000  --> Corn

* Total acreage used: 185 + 145 + 105 + 65 = 500 = MAX Available


# Planting Costs: 
* G[0] = 112100.00000


# Expected Yields:
* E[0] = 362.50000     --> no need to buy any wheat, excess of 362.5 - 200 = 162.5
* E[1] = 195.00000     --> **need to buy 240-195=45 tons of corn**
* E[2] = 5800.00000    --> **have below the quota of sugar beets so do not need to sale any at reduced price**

# amounts of feed crops to purchase:
Y[0] = 0.00000  --> wheat
Y[1] = 45.00000 --> corn (Matches above)

# Purchasing costs:
* R[0] = 9450.00000    --> should be 45*210=9,450, thus the answer checks out

# Tons of Excess crops:
W[0] = 162.50 --> wheat
W[1] 0.00000  --> Corn
W[2] 5800.00000 --> Beets at highest price
W[3] 0.00000    --> Beets at lowest price

# Profits from crop sales:
* H[0] = 236425.00000
* have 162 excess wheat so should make = 162*170=27,625
* have 5800 beets to sell so should make = 5800*36=208800
* Total profits: 27625 + 208880 = 236,425

# Objective Check
* Thus H - G - R = 236425 - 112100 -  9450 = 114875, which is what the given objective was thus at least the math works out. 


# <a id=solution2><span style="color:crimson"><center>Solution Discussion: Problem 3</center></a>
> From the results with Gurobi, the optimal lot sizes(X) and field allocations are **plant 145 acres of wheat in lot 2, plant 65 acres of corn in lot 4, plant 185 acres of sugar beets in lot 1 and 105 in lot 3**. This lot-sizing and field allocation scheme leads to an optimal profit value of **$\$$114875**. 

>	The cell above shows the solution in a condensed form and points out several things about the solution:

> 1. The given solution will require the purchase of 65 tons of corn due to the yield being below the minimum requirement. This leads to an extra purchasing cost of 9450. 
> 2. Due to there being below the quota of sugar beets yielded they can all be sold at the higher price point. 



In [4]:
# save the notebook as a pdf
# to_PDF("notebook_title")